required tmc data : TMC_Identification.csv

In [8]:
import os
import math
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import osm2gmns as og

osm2gmns, version 0.4.2


In [9]:
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [10]:
def Create_Boundary(node_csv):
    df = node_csv
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x_coord'], df['y_coord']))
    
    minx = gdf.bounds['minx'].min()
    miny = gdf.bounds['miny'].min()
    maxx = gdf.bounds['maxx'].max()
    maxy = gdf.bounds['maxy'].max()
    bbox = [minx,miny,maxx,maxy]
    return bbox

In [11]:
tmc_path = '/usr/local/home/ysx28/Desktop/GMNS/TMC2GMNS/TMC'
gmns_path = '/usr/local/home/ysx28/Desktop/GMNS/TMC2GMNS/GMNS'



osm_path = 'OSM/consolidated'

In [12]:
create_folder(gmns_path)
print('reading tmc data...')

tmc = pd.read_csv(tmc_path + os.sep + 'TMC_Identification.csv')
# tmc = tmc.sort_values(by=['road_order'])

'''build node.csv'''
print('converting tmc data into gmns format...')


node_csv = pd.DataFrame()
node_csv['name'] = None
node_csv['x_coord'] = None
node_csv['y_coord'] = None
node_csv['z_coord'] = None
node_csv['node_type'] = None
node_csv['ctrl_type'] = None
node_csv['zone_id'] = None
node_csv['parent_node_id'] = None
node_csv['geometry'] = None

for i in range(0,len(tmc)-1):
    if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
        node_csv = node_csv.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'x_coord'], \
                                    'y_coord': tmc.loc[i,'y_coord'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'x_coord'].astype(str) + " " + tmc.loc[i,'y_coord'].astype(str) +")"}, ignore_index=True)
    else:
        node_csv = node_csv.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'x_coord'], \
                                    'y_coord': tmc.loc[i,'y_coord'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'x_coord'].astype(str) + " " + tmc.loc[i,'y_coord'].astype(str) +")"}, ignore_index=True)
        node_csv = node_csv.append({'name': tmc.loc[i,'tmc']+'END',\
                                    'x_coord': tmc.loc[i,'end_longitude'], \
                                    'y_coord': tmc.loc[i,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'end_longitude'].astype(str) + " " + tmc.loc[i,'end_latitude'].astype(str) +")"}, ignore_index=True)


node_csv = node_csv.append({'name': tmc.loc[i+1,'tmc'],\
                                    'x_coord': tmc.loc[i+1,'x_coord'], \
                                    'y_coord': tmc.loc[i+1,'y_coord'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'x_coord'].astype(str) + " " + tmc.loc[i+1,'y_coord'].astype(str) +")"}, ignore_index=True)

node_csv = node_csv.append({'name': tmc.loc[i+1,'tmc']+'END',\
                                    'x_coord': tmc.loc[i+1,'end_longitude'], \
                                    'y_coord': tmc.loc[i+1,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'end_longitude'].astype(str) + " " + tmc.loc[i+1,'end_latitude'].astype(str) +")"}, ignore_index=True)

node_csv.index.name = 'node_id'

node_csv.index += 100000001 #index from 0

node_csv.to_csv(gmns_path + os.sep + '/node_tmc.csv')

reading tmc data...
converting tmc data into gmns format...


In [13]:
'''build link.csv'''
link_csv = pd.DataFrame()
link_csv['name'] = None
link_csv['from_node_id'] = None
link_csv['to_node_id'] = None
link_csv['directed'] = None
link_csv['geometry_id'] = None
link_csv['geometry'] = None
link_csv['dir_flag'] = None
link_csv['parent_link_id'] = None
link_csv['length'] = None
link_csv['grade'] = None
link_csv['facility_type'] = None
link_csv['capacity'] = None
link_csv['free_speed'] = None
link_csv['lanes'] = None

for i in range(0,len(tmc)):
    link_csv = link_csv.append({'name': tmc.loc[i,'tmc'],\
                                    'from_node_id': node_csv[(node_csv['x_coord']==tmc.loc[i,'x_coord']) & (node_csv['y_coord']==tmc.loc[i,'y_coord'])].index.values[0], \
                                    'to_node_id': node_csv[(node_csv['x_coord']==tmc.loc[i,'end_longitude']) & (node_csv['y_coord']==tmc.loc[i,'end_latitude'])].index.values[0],\
                                    'directed': 1,\
                                    'geometry_id': None,\
                                    'geometry': "LINESTRING (" + tmc.loc[i,'x_coord'].astype(str) + " " + tmc.loc[i,'y_coord'].astype(str) + "," +\
                                        tmc.loc[i,'end_longitude'].astype(str) + tmc.loc[i,'end_latitude'].astype(str) + ")",\
                                    'dir_flag': 1,\
                                    'parent_link_id': None,\
                                    'length': tmc.loc[i,'miles'],\
                                    'grade': None,\
                                    'facility_type': 'interstate' if tmc.loc[i,'road'][0] == 'I'else None ,\
                                    'capacity':None,\
                                    'free_speed':None,\
                                    'lanes': None}, ignore_index=True)
link_csv.index.name = 'link_id'
link_csv.index += 100000001


link_csv.to_csv(gmns_path + os.sep + '/link_tmc.csv')


In [14]:
link_csv

,name,from_node_id,to_node_id,directed,geometry_id,geometry,dir_flag,parent_link_id,length,grade,facility_type,capacity,free_speed,lanes
link_id,,,,,,,,,,,,,,
100000001,110P04102,100000001,100000002,1,None,"LINESTRING (-77.10289 39.01887,-77.1050739.021...",1,None,0.190071,None,interstate,None,None,None
100000002,110+04103,100000002,100000003,1,None,"LINESTRING (-77.10507 39.021029999999996,-77.1...",1,None,1.215464,None,interstate,None,None,None
100000003,110P04103,100000003,100000004,1,None,"LINESTRING (-77.12297 39.03014,-77.12763000000...",1,None,0.256706,None,interstate,None,None,None
100000004,110+04104,100000004,100000005,1,None,"LINESTRING (-77.12763000000001 39.03093,-77.13...",1,None,0.563975,None,interstate,None,None,None
100000005,110P04104,100000005,100000006,1,None,"LINESTRING (-77.13789 39.03257,-77.1453539.0378)",1,None,0.560561,None,interstate,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000066,110N04104,100000067,100000068,1,None,"LINESTRING (-77.14494 39.0363,-77.1373939.03233)",1,None,0.533999,None,interstate,None,None,None
100000067,110-04103,100000068,100000069,1,None,"LINESTRING (-77.13739 39.03233,-77.12834000000...",1,None,0.497398,None,interstate,None,None,None
100000068,110N04103,100000069,100000070,1,None,"LINESTRING (-77.12834000000001 39.03089,-77.12...",1,None,0.269579,None,interstate,None,None,None


In [15]:
tmc

,tmc,road,direction,intersection,state,county,zip,y_coord,x_coord,end_latitude,end_longitude,miles,road_order,timezone_name,type,country,active_start_date,active_end_date
0,110P04102,I-270,NORTHBOUND,I-495/MD-355,MD,MONTGOMERY,20814,39.01887,-77.10289,39.02103,-77.10507,0.190071,1,America/New_York,P1.1,USA,2019-10-31 15:30:00-04:00,NaN
1,110+04103,I-270,NORTHBOUND,MD-187/OLD GEORGETOWN RD/EXIT 1,MD,MONTGOMERY,20814,39.02103,-77.10507,39.03014,-77.12297,1.215464,2,America/New_York,P1.3,USA,2019-10-31 15:30:00-04:00,NaN
2,110P04103,I-270,NORTHBOUND,MD-187/OLD GEORGETOWN RD/EXIT 1,MD,MONTGOMERY,20814,39.03014,-77.12297,39.03093,-77.12763,0.256706,3,America/New_York,P1.3,USA,2019-10-31 15:30:00-04:00,NaN
3,110+04104,I-270,NORTHBOUND,I-270-SPUR,MD,MONTGOMERY,20817,39.03093,-77.12763,39.03257,-77.13789,0.563975,4,America/New_York,P1.1,USA,2019-10-31 15:30:00-04:00,NaN
4,110P04104,I-270,NORTHBOUND,I-270-SPUR,MD,MONTGOMERY,20817,39.03257,-77.13789,39.03780,-77.14535,0.560561,5,America/New_York,P1.1,USA,2019-10-31 15:30:00-04:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,110N04104,I-270,SOUTHBOUND,I-270-SPUR,MD,MONTGOMERY,20817,39.03630,-77.14494,39.03233,-77.13739,0.533999,31,America/New_York,P1.1,USA,2019-10-31 15:30:00-04:00,NaN
66,110-04103,I-270,SOUTHBOUND,MD-187/OLD GEORGETOWN RD/EXIT 1,MD,MONTGOMERY,20814,39.03233,-77.13739,39.03089,-77.12834,0.497398,32,America/New_York,P1.3,USA,2019-10-31 15:30:00-04:00,NaN
67,110N04103,I-270,SOUTHBOUND,MD-187/OLD GEORGETOWN RD/EXIT 1,MD,MONTGOMERY,20814,39.03089,-77.12834,39.03001,-77.12346,0.269579,33,America/New_York,P1.3,USA,2019-10-31 15:30:00-04:00,NaN
68,110-04102,I-270,SOUTHBOUND,I-495/MD-355,MD,MONTGOMERY,20814,39.03001,-77.12346,39.02148,-77.10605,1.176331,34,America/New_York,P1.1,USA,2019-10-31 15:30:00-04:00,NaN


In [16]:
'''build link.csv'''
node_csv


,name,x_coord,y_coord,z_coord,node_type,ctrl_type,zone_id,parent_node_id,geometry
node_id,,,,,,,,,
100000001,110P04102,-77.10289,39.01887,None,tmc_start,None,None,None,POINT (-77.10289 39.01887)
100000002,110+04103,-77.10507,39.02103,None,tmc_start,None,None,None,POINT (-77.10507 39.021029999999996)
100000003,110P04103,-77.12297,39.03014,None,tmc_start,None,None,None,POINT (-77.12297 39.03014)
100000004,110+04104,-77.12763,39.03093,None,tmc_start,None,None,None,POINT (-77.12763000000001 39.03093)
100000005,110P04104,-77.13789,39.03257,None,tmc_start,None,None,None,POINT (-77.13789 39.03257)
...,...,...,...,...,...,...,...,...,...
100000068,110-04103,-77.13739,39.03233,None,tmc_start,None,None,None,POINT (-77.13739 39.03233)
100000069,110N04103,-77.12834,39.03089,None,tmc_start,None,None,None,POINT (-77.12834000000001 39.03089)
100000070,110-04102,-77.12346,39.03001,None,tmc_start,None,None,None,POINT (-77.12346 39.03001)


In [17]:
import os
import math
import heapq
import numpy as np
import pandas as pd


# WGS84 transfer coordinate system to distance: meter
def LLs2Dist(lon1, lat1, lon2, lat2):
    R = 6371
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
    a = math.sin(dLat / 2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180.0) * math.cos(lat2 * math.pi / 180.0) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    dist = R * c * 1000
    return dist


def createConnector(dataList,from_node_id, to_node_id):
    link = []
    from_node_id_x = dataList[from_node_id]['x_coord']
    from_node_id_y = dataList[from_node_id]['y_coord']
    to_node_id_x = dataList[to_node_id]['x_coord']
    to_node_id_y = dataList[to_node_id]['y_coord']
    length = LLs2Dist(from_node_id_x,from_node_id_y,to_node_id_x,to_node_id_y)
    geometry = 'LINESTRING (' + str(from_node_id_x)+' '+str(from_node_id_y)+', '+str(to_node_id_x)+' '+str(to_node_id_y)+')'
        
    link = [from_node_id, to_node_id, length, geometry]
    return link




print('creating connector between osm network and gtfs data...')

node_tmc = pd.read_csv(gmns_path + os.sep +'/node_tmc.csv',low_memory=False) # transit node
node_road = pd.read_csv(osm_path + os.sep +'/node.csv', encoding='gbk',low_memory=False)

node_combine = pd.DataFrame()
node_combine['node_id']= node_road['node_id'].tolist() + node_tmc['node_id'].tolist()
node_combine['name']= node_road['name'].tolist() + node_tmc['name'].tolist()
node_combine['x_coord']= node_road['x_coord'].tolist() + node_tmc['x_coord'].tolist()
node_combine['y_coord'] = node_road['y_coord'].tolist() + node_tmc['y_coord'].tolist()
node_combine['node_type']= node_road['node_type'].tolist() + node_tmc['node_type'].tolist()


node_combine.to_csv(gmns_path + os.sep +'/node_combine.csv', index=False)
    
link_road = pd.read_csv(osm_path + os.sep +'/link.csv', encoding='gbk',low_memory=False)
link_road = link_road.drop(['link_id', 'osm_way_id', 'from_biway'], axis=1)
    
dataList_tmc = {}
gp = node_tmc.groupby('node_id')
for key, form in gp:
    dataList_tmc[key] = {
        'x_coord': form['x_coord'].values[0],
        'y_coord': form['y_coord'].values[0]
        }

dataList_road = {}
gp = node_road.groupby('node_id')
for key, form in gp:
    dataList_road[key] = {
        'x_coord': form['x_coord'].values[0],
        'y_coord': form['y_coord'].values[0]
        }

coord_list = []
for key in dataList_road.keys(): 
    coord_list.append((dataList_road[key]['x_coord'],dataList_road[key]['y_coord']))
coord_array = np.array(coord_list)


dataList = {}  #road node+tmc node
gp = node_combine.groupby('node_id')
for key, form in gp:
    dataList[key] = {
        'x_coord': form['x_coord'].values[0],
        'y_coord': form['y_coord'].values[0]
        }

# print('building connector...')
link_list = []

for key in dataList_tmc.keys(): 
    coord = np.array((dataList_tmc[key]['x_coord'],dataList_tmc[key]['y_coord']))
    coord_diff = coord_array - coord
    coord_diff_square = np.power(coord_diff,2)
    coord_diff_sum_square = coord_diff_square.sum(axis=1)
    distance = np.sqrt(coord_diff_sum_square)  #distance array between one tmc node to all road node 
    
    count = 1
    while (count):
        idx_temp = heapq.nsmallest(count, distance.tolist())
        idx = np.where(distance == idx_temp[count-1])  #index of the nearest road node
        active_node = node_road['node_id'].iloc[idx[0]]  #id of the nearest road node
        if ((active_node[idx[0][0]] in link_road['from_node_id'].tolist()) and (active_node[idx[0][0]] in link_road['to_node_id'].tolist())): #check the road node is the from/to node of road link 
            active_link1 = createConnector(dataList,active_node[idx[0][0]], key) #create link between the nearest road node and the selected tmc node
            active_link2= createConnector(dataList,key, active_node[idx[0][0]])
            link_list.append(active_link1)
            link_list.append(active_link2)
            break
        else:
            count += 1


connector_csv = pd.DataFrame()
connector_csv = pd.DataFrame(link_list, columns=['from_node_id','to_node_id','length','geometry']).drop_duplicates()    


connector_csv['name'] = None
connector_csv['link_type'] = None
connector_csv['link_type_name'] = 'connector'
connector_csv['dir_flag'] = 1
connector_csv['lanes'] = 1
connector_csv['free_speed'] = None
connector_csv['capacity'] = None
connector_csv['allowed_uses'] = None



combined_link = pd.concat([link_road,connector_csv], axis=0, ignore_index=True)

combined_link.index.name = 'link_id'
combined_link.index += 1

combined_link.to_csv(gmns_path + os.sep +'/link_osm_connector.csv')



creating connector between osm network and gtfs data...


In [18]:
combined_link

,name,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses
link_id,,,,,,,,,,,,
1,NaN,18031,17805,1,549.992364,4,59,NaN,motorway,1,"LINESTRING (-77.2548917 39.1847785, -77.255373...",auto
2,Schaeffer Road,21600,21604,1,235.763640,2,29,NaN,tertiary,5,"LINESTRING (-77.3012729 39.1500298, -77.300997...",auto
3,Schaeffer Road,21604,21606,1,127.063493,2,29,NaN,tertiary,5,"LINESTRING (-77.2989717 39.1511593, -77.298231...",auto
4,Schaeffer Road,21606,24184,1,115.330528,2,29,NaN,tertiary,5,"LINESTRING (-77.29760829999999 39.1515928, -77...",auto
5,Schaeffer Road,24184,28296,1,27.264078,2,29,NaN,tertiary,5,"LINESTRING (-77.2964717 39.1521341, -77.296288...",auto
...,...,...,...,...,...,...,...,...,...,...,...,...
63788,None,100000070,4640,1,78.068205,1,None,None,connector,None,"LINESTRING (-77.12346 39.03001, -77.123474 39....",None
63789,None,17013,100000071,1,13.208998,1,None,None,connector,None,"LINESTRING (-77.10600090000001 39.0213675, -77...",None
63790,None,100000071,17013,1,13.208998,1,None,None,connector,None,"LINESTRING (-77.10605 39.02148, -77.1060009000...",None
